# 1. Pacotes e Bibliotecas

In [ ]:
# Instalações
!pip install praw

import praw

In [ ]:
## Credenciais de acesso à API
# Login: wilil26564@wlmycn.com
# Senha: b@s)%fn0>k<JnTsx

secret = {'client_id':'DDb-ntFm5MMVyLGJ0UuQAw'
,'client_secret':'Aqk999rbxD8y2q7SQJG94x_KTeCVoQ'
,'user_agent':'script by u/UNICAMP2023'
,'username':'UNICAMP2023'
,'password':'b@s)%fn0>k<JnTsx'}

# Conexão com a API
reddit = praw.Reddit(**secret, warn_on_requests=False)

In [ ]:
# Importações
import pandas as pd  # Manipula estruturas de dados
import numpy as np  # Operações matemáticas

from datetime import datetime  # Manipulação temporal
import pytz  # Fuso horários
import requests  # Manipulação HTTP
import re  # Expressões regulares
from tqdm import tqdm  # Barra de progresso

import warnings  # Tratamento de alertas
warnings.filterwarnings('ignore')

# 2. Funções

In [ ]:
def get_author_info(author):
    # Obtém informações sobre o autor do registro (post ou comentário)
    try:
        author_info = requests.get(f"https://www.reddit.com/user/{author}/about.json", headers={"User-agent": "Mozilla/5.0"})
        if author_info.status_code == 200:
            author_data = author_info.json().get('data', {})
            karma = author_data.get('comment_karma')
        else:
            karma = np.nan
    except AttributeError:
        karma = np.nan
    return karma

def process_item(post, comment=None, sn=None):
    # Processa a resposta da API
    # sn = subreddit_name
    if comment:
        if isinstance(comment, praw.models.MoreComments):
            return np.nan
        author = comment.author.name if comment.author else '[deleted]'
        datetime_utc = comment.created_utc
        text = comment.body
    else:
        author = post.author.name if post.author else '[deleted]'
        datetime_utc = post.created_utc
        text = post.selftext if post.selftext else post.title

    item_data = {
        'register_index': str(post.id) + str('post' if comment is None else comment.id),
        'post_id': post.id,
        'comment_id': np.nan if comment is None else comment.id,
        'author': author,
        'datetime': datetime.fromtimestamp(datetime_utc),
        'title': np.nan if comment else post.title,
        'url': f"https://www.reddit.com/r/{sn}/comments/{post.id}/" if comment is None else np.nan,
        'score': post.score if comment is None else comment.score ,
        'comments': post.num_comments if comment is None else np.nan,
        'text': text,
        'author_post_karma': get_author_info(author),
        'tag': post.link_flair_text,
    }
    return item_data

# 3. Importação

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/UNICAMP/Mineração de Dados/ProjetoMineracao/data.csv', sep=',').iloc[:,:12]

# 4. Processamento

In [ ]:
# Define o nome do subreddit
sub = 'technology'

# Seleciona o subreddit
subreddit = reddit.subreddit(sub)

# Obtém os últimos 10 posts da plataforma
posts = subreddit.new(limit=10)

# Cria uma lista vazia para armazenar os resultados
df = []
new_topics = 0
new_comments = 0

# Itera sobre cada um dos últimos posts
for post in tqdm(posts):
    p = process_item(post, sn=sub)
    df.append(p)
    if p['register_index'] not in data['register_index'].values:
        new_topics += 1

    # Obtém os posts da plataforma
    post_comments = post.comments.list()

    # Itera sobre os comentários do post
    for comment in post_comments:
        c = process_item(post, comment, sn=sub)
        df.append(c)
        try:
            if c['register_index'] not in data['register_index'].values:
                new_comments += 1
        except:
            pass

0it [00:00, ?it/s]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

1it [00:01,  1.53s/it]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

2it [00:02,  1.11s/it]WARNING:praw:It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use A

In [ ]:
# Transforma a lista obtida em um DataFrame
data_in = []
for row,i in enumerate(df):
    if row == 0:
        data_in = pd.DataFrame.from_dict([i])
    else:
        # União com a base original
        data_in = pd.concat([data_in, pd.DataFrame.from_dict([i])], ignore_index=True)

# Remoção de valores nulos da extração
data_in = data_in[data_in['post_id'].notna()]

# Remove todos os caracteres especiais
data_in['text'] = data_in['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))

In [ ]:
# Remove informações duplicadas
data = data[~data['register_index'].isin(data_in['register_index'].values)]
data = pd.concat([data, data_in], ignore_index=True)

# 5. Resultados

In [ ]:
# Visualização preliminar da base
data.head()

,register_index,post_id,comment_id,author,datetime,title,url,score,comments,text,author_post_karma,tag
0,1691wy8post,1691wy8,NaN,West-Ad7245,2023-09-03 16:52:41,DJI Mini 3 Pro (DJI RC) + Fly More Kit Plus $8...,https://www.reddit.com/r/technology/comments/1...,16.0,6.0,DJI Mini 3 Pro DJI RC Fly More Kit Plus 8...,NaN,Robotics/Automation
1,1691wy8jyz4h35,1691wy8,jyz4h35,CrashingAtom,2023-09-03 17:35:50,NaN,NaN,25.0,NaN,Once again the technology sub has just become ...,NaN,Robotics/Automation
2,1691wy8jyz1kik,1691wy8,jyz1kik,AnomalousBean,2023-09-03 17:19:50,NaN,NaN,2.0,NaN,https media giphy com media KBaxHrT7rkeW5ma7...,NaN,Robotics/Automation
3,1691wy8jyzahzk,1691wy8,jyzahzk,Bobthebrain2,2023-09-03 18:10:57,NaN,NaN,5.0,NaN,You get an upvote OP gets a downvote It s al...,NaN,Robotics/Automation
4,169180gjyzb77k,169180g,jyzb77k,hail_satine,2023-09-03 18:14:57,NaN,NaN,5.0,NaN,Eat glass you sanctimonious trash bag full of ...,NaN,Politics


In [ ]:
# Resumo da extração em comparação à base principal
rows_summary = pd.DataFrame({
    'Registros anterioes': data.shape[0],
    'Novos registros': new_topics + new_comments,
    'Total': data.shape[0] + new_topics + new_comments,
    'Novos posts': new_topics,
    'Novos comentários': new_comments
}, index=[0]).T
rows_summary.columns = ['Count']
rows_summary

,Count
Registros anterioes,9433
Novos registros,88
Total,9521
Novos posts,1
Novos comentários,87


# 6. Exportação

In [ ]:
# Salva base de dados atualizada
data.iloc[:,:12].to_csv('/content/drive/MyDrive/UNICAMP/Mineração de Dados/ProjetoMineracao/data.csv', index=False, sep=',')